# Finetuning falcon-1b with Axolotl+QLoRA

This notebook makes it easy to try out finetuning falcon-1b with Axolotl+QLoRA on LambdaLabs

If you run into any issues, welcome to report [here](https://github.com/OpenAccess-AI-Collective/axolotl/pull/132) .

In [8]:
%env DEBIAN_FRONTEND=noninteractive

env: DEBIAN_FRONTEND=noninteractive


In [2]:
!python -V

Python 3.8.10


In [3]:
!pip -V

pip 23.0.1 from /home/ubuntu/.local/lib/python3.8/site-packages/pip (python 3.8)


In [5]:
!whoami

ubuntu


In [10]:
!sudo apt install -yq python3.10

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10-minimal
Suggested packages:
  python3.10-venv binfmt-support
The following NEW packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10 python3.10-minimal
0 upgraded, 4 newly installed, 0 to remove and 28 not upgraded.
Need to get 5225 kB of archives.
After this operation, 20.2 MB of additional disk space will be used.
Err:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.10-minimal amd64 3.10.10-1+focal1
  404  Not Found [IP: 185.125.190.52 443]
Err:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.10-minimal amd64 3.10.10-1+focal1
  404  Not Found [IP: 185.125.190.52 443]
Err:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.10-stdlib amd64 3.10.10-1+focal1
  404  Not F

In [12]:
!sudo apt install -yq python3.9
!sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.9 1
!sudo update-alternatives --config python # pick 3.9 if given option
!python -V # should be 3.9

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpython3.9-minimal libpython3.9-stdlib python3.9-minimal
Suggested packages:
  python3.9-venv binfmt-support
The following NEW packages will be installed:
  libpython3.9-minimal libpython3.9-stdlib python3.9 python3.9-minimal
0 upgraded, 4 newly installed, 0 to remove and 28 not upgraded.
Need to get 5023 kB of archives.
After this operation, 19.7 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.9-minimal amd64 3.9.16-1+focal1 [804 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.9-minimal amd64 3.9.16-1+focal1 [2062 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.9-stdlib amd64 3.9.16-1+focal1 [1662 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu focal/main amd64 pyt

In [13]:
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

--2023-06-03 15:42:40--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.192.175, 151.101.128.175, 151.101.64.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.192.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2578580 (2.5M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.46M  --.-KB/s    in 0.009s  

2023-06-03 15:42:41 (284 MB/s) - ‘get-pip.py’ saved [2578580/2578580]

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 148.7 MB/s eta 0:00:00


In [14]:
!pip3 install torch --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118


In [15]:
!pip3 list|grep torch

torch                  1.13.1
torchvision            0.14.1


In [ ]:
!pip3 install -U torch --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118


In [18]:
!git clone https://github.com/OpenAccess-AI-Collective/axolotl

Cloning into 'axolotl'...
remote: Enumerating objects: 2526, done.
remote: Counting objects: 100% (1269/1269), done.
remote: Compressing objects: 100% (385/385), done.
remote: Total 2526 (delta 923), reused 1100 (delta 818), pack-reused 1257
Receiving objects: 100% (2526/2526), 1.30 MiB | 12.59 MiB/s, done.
Resolving deltas: 100% (1562/1562), done.


In [19]:
%cd axolotl

/home/ubuntu/axolotl


In [20]:
!pip3 install -e .

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/ubuntu/axolotl
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-mt8imf30/transformers_e7d69891845841ad93145f052bc3f2f5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-mt8imf30/transformers_e7d69891845841ad93145f052bc3f2f5
  Resolved https://github.com/huggingface/transformers.git to commit 539e2281cd97c35ef4122757f26c88f44115fa94
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 kB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [21]:
!accelerate config --config_file configs/accelerate/default_config.yaml default

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Traceback (most recent call last):
  File "/home/ubuntu/.local/bin/accelerate", line 5, in <module>
    from accelerate.commands.accelerate_cli import main
  File "/home/ubuntu/.local/lib/python3.9/site-packages/accelerate/__init__.py", line 3, in <module>
    from .accelerator import Accelerator
  File "/home/ubuntu/.local/lib/python3.9/site-packages/accelerate/accelerator.py", line 39, in <module>
    from .tracking import LOGGER_TYPE_TO_CLASS, GeneralTracker, filter_trackers
  File "/home/ubuntu/.local/lib/python3.9/site-packages/accelerate/tracking.py", line 42, in <module>
    from torch.utils import tensorboard
  File "/home/ubuntu/.local/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py", line 12, in <module>
    from .writer import FileWrite

In [22]:
!pwd

/home/ubuntu/axolotl


In [24]:
!pip3 install -e . # change depend on needs
!pip3 install protobuf==3.20.3
!pip3 install -U requests
!pip3 install -U --ignore-installed psutil
!pip3 install -U scipy
!pip3 install git+https://github.com/huggingface/peft.git # not for gptq

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/ubuntu/axolotl
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-48bo5v0h/transformers_137884897c0d4fd8ba70023bd1155185
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-48bo5v0h/transformers_137884897c0d4fd8ba70023bd1155185
  Resolved https://github.com/huggingface/transformers.git to commit 539e2281cd97c35ef4122757f26c88f44115fa94
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Attempting uninstall: axolotl
    Found existing installation: axolotl 0.1
    Uninstalling axolotl-0.1:
      Successfully uninstalled axolotl-0.1
  Running setup.py develop for axolotl
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━

In [25]:
!echo $LD_LIBRARY_PATH

In [26]:
%env LD_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu

env: LD_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu


In [29]:
!accelerate config default

accelerate configuration saved at /home/ubuntu/.cache/huggingface/accelerate/default_config.yaml


In [31]:
!wget https://raw.githubusercontent.com/utensil/axolotl/falcon-7b-qlora/examples/falcon/config-7b-qlora.yml

--2023-06-03 15:51:47--  https://raw.githubusercontent.com/utensil/axolotl/falcon-7b-qlora/examples/falcon/config-7b-qlora.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2238 (2.2K) [text/plain]
Saving to: ‘config-7b-qlora.yml’

config-7b-qlora.yml 100%[===================>]   2.19K  --.-KB/s    in 0s      

2023-06-03 15:51:47 (102 MB/s) - ‘config-7b-qlora.yml’ saved [2238/2238]



In [33]:
!cp config-7b-qlora.yml config-1b-qlora.yml
!sed -i -e 's/falcon-7b/falcon-rw-1b/g' -e 's/wandb_project: falcon-qlora/wandb_project: /g' -e 's/bf16: true/bf16: false/g'  -e 's/tf32: true/tf32: false/g' config-1b-qlora.yml

In [34]:
!cat config-1b-qlora.yml

# 1b: tiiuae/falcon-rw-1b
# 40b: tiiuae/falcon-40b
base_model: tiiuae/falcon-rw-1b
base_model_config: tiiuae/falcon-rw-1b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-rw-1b/tree/main 
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false
push_dataset_to_hub:
datasets:
  - path: QingyiSi/Alpaca-CoT
    data_files:
      - Chain-of-Thought/formatted_cot_data/gsm8k_train.json
    type: "alpaca:chat"
dataset_prepared_path: last_run_prepared
val_set_size: 0.01
# enable QLoRA
adapter: qlora
lora_model_dir:
sequence_len: 2048
max_packed_sequence_len:

# hyperparameters from QLoRA paper Appendix B.2
# "We find hyperparameters to be largely robust across datasets"
lora_r: 64
lora_alpha: 16
# 0.1 for models up to 13B
# 0.05 for 33B and 65B models
lora_dropout: 0.05
# add LoRA modules on all linear layers of the base model
lora_target_modules:
l

In [35]:
%env WANDB_MODE=offline

env: WANDB_MODE=offline


In [36]:
!accelerate launch scripts/finetune.py config-1b-qlora.yml


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/lib/x86_64-linux-gnu/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/ubuntu/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...
INFO:root:loading tokenizer... tiiuae/falcon-rw-1b
INFO:filelock:Lock 139620631419392 acquired on /home/ubuntu/.cache/huggingface/hub/models--tiiuae--falcon-rw-1b/blobs/31a4ebf2104713af5922ab75c18e61597a6f076e.lock
INFO:filelock:Lock 139620631419392 released on /home/ubuntu/.cache/huggingface/hub/models--tiiuae--falcon-rw-1b/blobs

In [37]:
# https://github.com/explosion/spaCy/issues/7160
!pip3 install click --upgrade

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 29.5 MB/s eta 0:00:00


In [38]:
!accelerate launch scripts/finetune.py config-1b-qlora.yml


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/lib/x86_64-linux-gnu/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/ubuntu/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...
INFO:root:loading tokenizer... tiiuae/falcon-rw-1b
Using pad_token, but it is not set yet.
INFO:root:Loading prepared dataset from disk at last_run_prepared/0ecc5b78e3ce4254b22e749b093712b4...
INFO:root:Prepared dataset loaded from disk...
INFO:root:loading model and peft_config...
INFO:root:converting PEFT model w/ prepare_model_